## Imports and Setup

In [1]:
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import os 
os.chdir('/home/itingzon.unicef/giga')
from src import sat_download
from utils import post_utils
from utils import calib_utils
from utils import config_utils
from utils import data_utils

%load_ext autoreload
%autoreload 2

## Load Config Files

In [2]:
iso_code = "SEN"
sum_threshold=0
buffer_size=0
calibrator=None
cam_method="gradcam"

data_config_file = os.path.join(os.getcwd(), "configs/data_configs/data_config_ISO_AF.yaml")
model_config_file = os.path.join(os.getcwd(), "configs/best_models.yaml")
data_config = config_utils.load_config(data_config_file)
model_config = config_utils.load_config(model_config_file)

## Load Model Predictions

In [4]:
preds = post_utils.load_preds(
    iso_code, data_config, model_config, cam_method, sum_threshold, buffer_size, calibrator
)
preds.head(2)

Reading files for SEN...: 100%|████████████████████| 45/45 [00:03<00:00, 14.39it/s]                                     
INFO:root:Data dimensions: (15203, 9)


,prob,UID,sum,group,geometry,ADM1,ADM2,ADM3,PUID
0,0.966634,23507,22.0,8567,POINT (-1814593.415 1665931.749),Diourbel,Diourbel,Ndindy,Diourbel_23507
1,0.966054,178498,15.0,14124,POINT (-1811869.892 1464003.347),Ziguinchor,Bignona,Sindian,Bignona_178498


## Load Government Data

In [4]:
master = post_utils.load_master(iso_code, data_config)
master.head(2)

INFO:root:clean
0    9033
2     794
1     238
Name: count, dtype: int64
INFO:root:Data dimensions: (10065, 13)


,MUID,source,iso,country,region,subregion,name,giga_id_school,clean,geometry,ADM1,ADM2,ADM3
0,UNICEF-SEN-SCHOOL-00000000,UNICEF,SEN,Senegal,Africa,Sub-Saharan Africa,EE ALIEU SAMB,7614c6c6-9aca-36ff-978b-22bfca59916a,0,POINT (-17.51261 14.74680),Dakar,Dakar,Almadies
1,UNICEF-SEN-SCHOOL-00000001,UNICEF,SEN,Senegal,Africa,Sub-Saharan Africa,EE EL HADJI BAYTIR SAMB,350fb172-bfef-331d-bd55-5c51139b9f48,0,POINT (-17.51196 14.74951),Dakar,Dakar,Almadies


## Load OSM and Overture Data

In [5]:
osm_overture = post_utils.load_osm_overture(iso_code, data_config)
osm_overture.head(2)

INFO:root:clean
0    784
Name: count, dtype: int64
INFO:root:Data dimensions: (784, 13)


,SUID,source,iso,country,region,subregion,name,giga_id_school,clean,geometry,ADM1,ADM2,ADM3
0,OSM-SEN-SCHOOL-00000000,OSM,SEN,Senegal,Africa,Sub-Saharan Africa,Ecole Nationale de Police,None,0,POINT (-17.44890 14.68903),Dakar,Dakar,Plateau
1,OSM-SEN-SCHOOL-00000001,OSM,SEN,Senegal,Africa,Sub-Saharan Africa,CIFOP,None,0,POINT (-16.90105 15.16665),Thies,Tivaoune,Meouane


## Calculate Distances

In [6]:
preds, master, osm_overture = post_utils.calculate_nearest_distances(preds, master, osm_overture)
preds.head(2)

,prob,UID,sum,group,geometry,ADM1,ADM2,ADM3,PUID,distance_to_nearest_master,MUID,distance_to_nearest_osm_overture,SUID
0,0.966634,23507,22.0,8567,POINT (-16.30077 14.79800),Diourbel,Diourbel,Ndindy,Diourbel_23507,23.830582,UNICEF-SEN-SCHOOL-00006044,20329.970076,OSM-SEN-SCHOOL-00000627
1,0.966054,178498,15.0,14124,POINT (-16.27630 13.03738),Ziguinchor,Bignona,Sindian,Bignona_178498,18.696889,UNICEF-SEN-SCHOOL-00001335,12728.227840,OSM-SEN-SCHOOL-00000081


## Save Results

In [7]:
post_utils.save_results(iso_code, preds, model_config, cam_method=cam_method, source="preds")
post_utils.save_results(iso_code, master, model_config, source="master")
post_utils.save_results(iso_code, osm_overture, model_config, source="osm_overture")